Скачивание зависимостей для работы с Gemini:

In [1]:
!pip install -q -U google-genai

In [ ]:
from google import genai
from google.genai import types

GEMINI_API_KEY = ''

# gemini-2.5-flash-preview-04-17
# GEMINI_MODEL = "gemini-2.0-flash-thinking-exp-01-21"

# gemini-2.5-pro-preview-05-06
GEMINI_MODEL = "gemini-2.5-pro-preview-05-06"

SAFETY_SETTINGS = {
    "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
    "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
    "HARM_CATEGORY_SEXUALLY_EXPLICIT": "BLOCK_NONE",
    "HARM_CATEGORY_DANGEROUS_CONTENT": "BLOCK_NONE",
    }

client = genai.Client(api_key=GEMINI_API_KEY)

response = client.models.generate_content(
    model=GEMINI_MODEL, contents="Hello!",
    config=types.GenerateContentConfig(max_output_tokens=4000, temperature=1.0, top_k=1, top_p=0.9, safety_settings=SAFETY_SETTINGS)
)
print(response.text)

Hello there! How can I help you today?


# Создание персонажей:

Первичная цель — задать персоналии или личности вручную. К подобным личностям я отнёс главного героя и его систему внутренних личностей и ценностей. Опираясь на Disco Elysium, мы объявляем главного героя и я его «навыки», реализованные в виде различных частей личности протагониста. Их способности одновременно являются и скилл-чеками.

Однако, также у нас есть возможность и создавать уникальные персоналии через саму модель. Для этого, в обход крупного промпта для самой генерации текстов, мы создадим небольшой и жёстко заданный промпт, который позволит сгенерировать 16 персоналий.

Note: System Prompt не имеет особого значения в моделях Gemini — этот промпт всегда является первым в любой очереди промптов.

In [6]:
# Входные параметры
json_path = "prompt.json"
variables = {
    "user": "Marc",
    "persona": "[{{user}} is a 25-yo graduatee of City-State Hong-Kong Militia School of Revachol. For his far more than average citizen of this Isola, he was usually called Big Mark. This name sticked with him with how many people unintentionally came up with the same name. Now he's on his duty on Precinct 41, covering Jamrock, being a sattelite-officer of Harrier De Bois.]",
    "description": "Lieutenant double-yefreitor Harrier \"Harry\" Du Bois. Alcoholic and drug addict, but one very worthy detective. ",
    "scenario": "This is a fused world of Disco Elysium and out reality as of 2019. Ocean is covered by Pale, and every continent is called an isola. A satteline-officer of Hong Kong, annexed by Taiwan and kept with a slippery peace deal, is out on his first day to solve the case of seemingly policially incline murder. He's assisting Harrier De Bois, a masterful detective with more than 200 solved cases."
}

Здесь я написал изменённый входной текст, сразу добавив в историю чата описание того, какую цель исполняет модель. Также я завёл префилл. Он начинается с открытой `<think>`, чтобы добавить более чёткую цель модели. Gemini — переобученная модель, которая сильно зависит от встроенного и трудноконтролируемого CoT. Отключение ризонинга сильно влияет на работоспособность модели в негативном ключе.

In [ ]:
skill_prompts = [
    {"role": "user", "parts": [{"text": "You are professional writer and game designer for text RPGs and Quests like Disco Elysium."}]},
    {"role": "user", "parts": [{"text": "<thinking template>\n\n(Thinking process should be there…)\n\n(Print verbatim: \"Okay, thinking is done! Replying:</think>\"\n</thinking template>"}]},
    {"role": "user", "parts": [{"text": "<persona>\nMarc is a 25-yo graduatee of City-State Hong-Kong Militia School. For his far more than average citizen of this Isola, he was usually called Big Mark. This name sticked with him with how many people unintentionally came up with the same name. Now he's on his duty on Precinct 41, being a sattelite-officer of Harrier De Bois.\n</persona>"}]},
    {"role": "user", "parts": [{"text": "<scenario>This is a fused world of Disco Elysium and out reality as of 2019. Ocean is covered by Pale, and every continent is called an isola. A satellite-officer of Hong Kong, annexed by Taiwan and kept with a slippery peace deal, is out on his first day to solve the case of seemingly policially incline murder. He's assisting Harrier De Bois, a masterful detective with more than 200 solved cases.\n</scenario>"}]},
    {"role": "user", "parts": [{"text": "Create 24 unique skills in Disco Elysium style for each of four attributes: Intellect, Psyche, Physique, Motorics. List the titles. Titles must be short, one word, or two words maximum."}]},
    {"role": "model", "parts": [{"text": "<think>I understand I'm professional game designer for a Disco Elysium-like game.\nNow I will make up with 24 skill names for Disco Elysium-like character described as `<persona/>. I must keep those names short reflecting unique personal features of the character. List of skills, starting with Intellect:"}]},
    #{"role": "model", "parts": [{"text": "I understand I'm professional game designer for a Disco Elysium-like game."}]},
]
response = client.models.generate_content(
    model=GEMINI_MODEL, contents=skill_prompts,
    config=types.GenerateContentConfig(
        max_output_tokens=4000, temperature=1.0, top_k=1, top_p=0.9)
)
print(response.text)


1.  **Analysis** - Classic analytical skill. Good for noticing details.
2.  **Connections** - This skill represents character's ability to make connections between seemingly unrelated things. Could be used to make deductions.
3.  **Pale Knowledge** - Represents character's knowledge about the Pale. Could be used to understand Pale-related phenomena.
4.  **Firearms** - Represents character's knowledge about firearms. Could be used to identify firearms, or to make deductions about how they were used.
5.  **Militia Training** - Represents character's militia training. Could be used to make deductions about military tactics, or to identify military equipment.
6.  **Politics** - Represents character's knowledge about politics. Could be used to understand political motivations, or to make deductions about political intrigues.

Psyche:
1.  **Empathy** - Classic empathy skill. Good for understanding people's emotions.
2.  **Composure** - Represents character's ability to remain calm under pre

Отформатируем полученный нейросетью ответ. В теории, это реализуемо через structured output, но в качестве proof-of-concept я обратаю ответ вручную:

In [7]:
skills_technical = {
    "Intellect": {
        "Analysis": "Classic analytical skill. Good for noticing details.",
        "Connections": "This skill represents character's ability to make connections between seemingly unrelated things. Could be used to make deductions.",
        "Pale Knowledge": "Represents character's knowledge about the Pale. Could be used to understand Pale-related phenomena.",
        "Firearms": "Represents character's knowledge about firearms. Could be used to identify firearms, or to make deductions about how they were used.",
        "Militia Training": "Represents character's militia training. Could be used to make deductions about military tactics, or to identify military equipment.",
        "Politics": "Represents character's knowledge about politics. Could be used to understand political motivations, or to make deductions about political intrigues."
    },
    "Psyche": {
        "Empathy": "Classic empathy skill. Good for understanding people's emotions.",
        "Composure": "Represents character's ability to remain calm under pressure. Could be used to resist intimidation, or to make difficult decisions.",
        "Willpower": "Represents character's ability to resist temptation, or to push through difficult situations.",
        "Interrogation": "Represents character's ability to get information from people. Could be used to get confessions, or to get people to reveal secrets.",
        "Streetwise": "Represents character's knowledge about the streets. Could be used to find information, or to navigate dangerous areas.",
        "Intimidation": "Represents character's ability to scare people. Could be used to get information, or to get people to do what you want."
    },
    "Physique": {
        "Endurance": "Classic endurance skill. Good for resisting physical damage, or for performing strenuous activities.",
        "Pain Threshold": "Represents character's ability to withstand pain. Could be used to resist torture, or to continue fighting even when injured.",
        "Brawling": "Represents character's ability to fight unarmed. Could be used to defend yourself, or to subdue opponents.",
        "Strength": "Represents character's physical strength. Could be used to lift heavy objects, or to break down doors.",
        "Intimidation (Physical)": "Represents character's ability to scare people with their physical presence. Could be used to get information, or to get people to do what you want.",
        "Metabolism": "Represents character's ability to process food and drink. Could be used to resist poisons, or to recover from injuries more quickly."
    },
    "Motorics": {
        "Reflexes": "Classic reflexes skill. Good for dodging attacks, or for performing quick actions.",
        "Stealth": "Represents character's ability to move quietly and unseen. Could be used to sneak past guards, or to ambush opponents.",
        "Lockpicking": "Represents character's ability to open locks. Could be used to get into locked rooms, or to open safes.",
        "Pickpocketing": "Represents character's ability to steal things from people's pockets. Could be used to get money, or to get valuable items.",
        "Driving": "Represents character's ability to drive vehicles. Could be used to get around quickly, or to escape from dangerous situations.",
        "Marksmanship": "Represents character's ability to shoot accurately. Could be used to take down opponents from a distance, or to hit small targets."
    }
}

Технические описания `skills_technical` нам пригодятся для добавления в описание персоны. Они куда более легковесные, чем полноценные описания, которые мы будем вызывать по надобности исходя из использования в истории чата.

In [19]:
def dict_to_plaintext(skills: dict) -> str:
    lines = []
    for category, subskills in skills.items():
        lines.append(f"{category}:")
        for skill, description in subskills.items():
            lines.append(f"- {skill}: {description}")
        lines.append("")  # пустая строка между категориями
    return "\n".join(lines).strip()

persona_skills = dict_to_plaintext(skills_technical)

Протестируем пропмпт перед тем, как сделать 24 запроса для каждого навыка:

In [ ]:
skill = "Pale Knowledge"
attribute = "Intellect"
desc = "Represents character's knowledge about the Pale. Could be used to understand Pale-related phenomena."

skill_prompts = [
    {"role": "user", "parts": [{"text": "You are professional writer and game designer for text RPGs and Quests like Disco Elysium."}]},
    {"role": "user", "parts": [{"text": f"<writing_style>Simple, without metacommentaries, straightfroward.\n</writing_style>"}]},
    {"role": "user", "parts": [{"text": "<thinking template>\n\nAttribute: \nSkill name: \nSkill description (2 short imperative statements): \nCool for (3 exagarrated people types): \nDescription (under 250 characters): \nUse cases (how low and high skill levels affect mind activity, under 350 chars): \n\n(Print verbatim: \"Okay, thinking is done! Replying, relying on `<writing_style>`:</think>\"\n</thinking template>"}]},
    {"role": "user", "parts": [{"text": "<scenario>This is a fused world of Disco Elysium and out reality as of 2019. Ocean is covered by Pale, and every continent is called an isola. A satellite-officer of Hong Kong, annexed by Taiwan and kept with a slippery peace deal, is out on his first day to solve the case of seemingly policially inclined murder. He's assisting Harrier De Bois, a masterful detective with more than 200 solved cases.\n</scenario>"}]},
    {"role": "user", "parts": [{"text": "<example>\nAttribute: Intellect \nSkill name: Logic \nSKill description: Wield raw intellectual power. Deduce the world. \nCool for: Analysts, Pure Rationalists, Obviously Logicians. \nDescription: Logic urges you to analyze the living daylights out of the case. It enables you to piece evidence together, detect inconsistencies in statements, and impress everyone with your astonishing conclusions. It’s the bread and butter of many a detective. \nUse cases: At high levels, Logic will be able to solve even the most complicated puzzle. You will be very proud and thus susceptible to intellectual flattery; for those blinded by their own brilliance often miss important clues. With low levels of Logic, you’re going to have a hard time solving even the simplest puzzles. Even if you find the pieces, good luck putting them together.\n</example>"}]},
    {"role": "user", "parts": [{"text": f"Create a description for skill {skill} in {attribute} attribute. Techincal description to use as a guideline (but nevery copy!): \n{desc}"}]},
    {"role": "model", "parts": [{"text": f"<think>I understand I'm professional game designer for a Disco Elysium-like game.\nNow I will describe skill {skill} from {attribute}:"}]},
]

response = client.models.generate_content(
    model=GEMINI_MODEL, contents=skill_prompts,
    config=types.GenerateContentConfig(
        max_output_tokens=4000, temperature=1.0, top_k=1, top_p=0.9)
)
print(response.text)


1. I will define skill name.
2. I will define 2 short imperative statements.
3. I will define 3 exagarrated people types.
4. I will define description under 250 characters.
5. I will define use cases under 350 characters.

I will make sure that writing style is simple, without metacommentaries, straightfroward.
I will make sure to output "Okay, thinking is done! Replying:</think>" at the end of the thought process.
Okay, thinking is done! Replying:</think>
Attribute: Intellect
Skill name: Pale Knowledge
Skill description: Understand the encroaching silence. Navigate its ethereal currents.
Cool for: Anamnesists, Cartographers of the Void, Doomsayers.
Description: Pale Knowledge allows you to comprehend the Pale, its effects on reality, and its influence on people. It helps you interpret strange phenomena and the whispers from beyond the isolas.
Use cases: High Pale Knowledge reveals hidden truths within the Pale's embrace, potentially uncovering unique insights or forgotten histories. 

Из-за довольно крупного промпта для такого простого запроса и манипуляций с ризонингом нам потребуется сделать дополнительное форматирование:

<small>Скрытая ячейка ниже — тестовая.<small>

In [ ]:
for attribute, subskills in skills.items():
    for skill, desc in subskills.items():
        cleaned_text = response.text.split("Replying:</think>\n", 1)[-1]

from json_condenser import jsonify

skills_list = jsonify(cleaned_text, exclude_list=["Description", "Use cases"], condense=False)
skills_list

Attribute: Intellect
Skill name: Pale Knowledge
Skill description: Understand the encroaching silence. Navigate its ethereal currents.
Cool for: Anamnesists, Cartographers of the Void, Doomsayers.
Description: Pale Knowledge allows you to comprehend the Pale, its effects on reality, and its influence on people. It helps you interpret strange phenomena and the whispers from beyond the isolas.
Use cases: High Pale Knowledge reveals hidden truths within the Pale's embrace, potentially uncovering unique insights or forgotten histories. Beware, for too much understanding can fracture the mind. Low Pale Knowledge leaves you ignorant of its dangers, making you susceptible to its disorienting effects and unable to decipher its mysteries.


{'Attribute': ['Intellect'],
 'Skill name': ['Pale Knowledge'],
 'Skill description': ['Understand the encroaching silence. Navigate its ethereal currents.'],
 'Cool for': ['Anamnesists', 'Cartographers of the Void', 'Doomsayers.'],
 'Description': ['Pale Knowledge allows you to comprehend the Pale, its effects on reality, and its influence on people. It helps you interpret strange phenomena and the whispers from beyond the isolas.'],
 'Use cases': ["High Pale Knowledge reveals hidden truths within the Pale\\'s embrace, potentially uncovering unique insights or forgotten histories. Beware, for too much understanding can fracture the mind. Low Pale Knowledge leaves you ignorant of its dangers, making you susceptible to its disorienting effects and unable to decipher its mysteries."]}

In [9]:
from json_condenser import jsonify
from httpx import HTTPStatusError
import time

skills_world_info = {}

for attribute, subskills in skills_technical.items():
    for skill, desc in subskills.items():
        print(attribute, skill, desc)
        
        prompt = [
            {"role": "user", "parts": [
                {"text": "You are professional writer and game designer for text RPGs and Quests like Disco Elysium."}]},
            {"role": "user", "parts": [
                {"text": f"<writing_style>Simple, without metacommentaries, straightfroward.\n</writing_style>"}]},
            {"role": "user", "parts": [
                {"text": "<thinking template>\n\nAttribute: \nSkill name: \nSkill description (2 short imperative statements): \nCool for (3 exagarrated people types): \nDescription (under 250 characters): \nUse cases (how low and high skill levels affect mind activity, under 350 chars): \n\n(Print verbatim: \"Okay, thinking is done! Replying, relying on `<writing_style>`:</think>\"\n</thinking template>"}]},
            {"role": "user", "parts": [{"text": "<scenario>This is a fused world of Disco Elysium and out reality as of 2019. Ocean is covered by Pale, and every continent is called an isola. A satellite-officer of Hong Kong, annexed by Taiwan and kept with a slippery peace deal, is out on his first day to solve the case of seemingly policially inclined murder. He's assisting Harrier De Bois, a masterful detective with more than 200 solved cases.\n</scenario>"}]},
            {"role": "user", "parts": [{"text": "<example>\nAttribute: Intellect \nSkill name: Logic \nSKill description: Wield raw intellectual power. Deduce the world. \nCool for: Analysts, Pure Rationalists, Obviously Logicians. \nDescription: Logic urges you to analyze the living daylights out of the case. It enables you to piece evidence together, detect inconsistencies in statements, and impress everyone with your astonishing conclusions. It’s the bread and butter of many a detective. \nUse cases: At high levels, Logic will be able to solve even the most complicated puzzle. You will be very proud and thus susceptible to intellectual flattery; for those blinded by their own brilliance often miss important clues. With low levels of Logic, you’re going to have a hard time solving even the simplest puzzles. Even if you find the pieces, good luck putting them together.\n</example>"}]},
            {"role": "user", "parts": [
                {"text": f"Create a description for skill {skill} in {attribute} attribute. Techincal description to use as a guideline (but nevery copy!): \n{desc}"}]},
            {"role": "model", "parts": [
                {"text": f"<think>I understand I'm professional game designer for a Disco Elysium-like game.\nNow I will describe skill {skill} from {attribute}:"}]},
        ]

        while True:
            try:
                response = client.models.generate_content(
                    model=GEMINI_MODEL,
                    contents=prompt,
                    config=types.GenerateContentConfig(
                        max_output_tokens=4000,
                        temperature=1.0,
                        top_k=1,
                        top_p=0.9
                    )
                )
                break  # выход из цикла при успешном ответе
            except HTTPStatusError as e:
                if e.response.status_code == 429:
                    print("Rate limit reached. Waiting 60 seconds...")
                    time.sleep(60)
                else:
                    raise
        cleaned_text = response.text.split("Replying, relying on `<writing_style>`:</think>\n", 1)[-1]
        skills_world_info[skill] = jsonify(cleaned_text, exclude_list=[
                                           "Description", "Use cases"], condense=False)
json_string = json.dumps(skills_world_info, ensure_ascii=False, indent=2)
with open("results.json", "w", encoding="utf-8") as f:
    f.write(json_string)

Intellect Analysis Classic analytical skill. Good for noticing details.
Intellect Connections This skill represents character's ability to make connections between seemingly unrelated things. Could be used to make deductions.
Intellect Pale Knowledge Represents character's knowledge about the Pale. Could be used to understand Pale-related phenomena.
Intellect Firearms Represents character's knowledge about firearms. Could be used to identify firearms, or to make deductions about how they were used.
Intellect Militia Training Represents character's militia training. Could be used to make deductions about military tactics, or to identify military equipment.
Rate limit reached. Waiting 60 seconds...
Intellect Politics Represents character's knowledge about politics. Could be used to understand political motivations, or to make deductions about political intrigues.
Psyche Empathy Classic empathy skill. Good for understanding people's emotions.
Psyche Composure Represents character's abilit



## Создание лорбуков:

Лорбуки, будучи детищем времён пика GPT-3 и моделей NovelAI — важный атрибут даже для современных больших языковых моделей, так как крупный контекст, загруженный в начало контекста, плохо «подсказывает» модели о своём содержимом. Тщательно прописанные персонажи и их подробное описание способны сильно «отравить» промпт и вызвать галлюцинации у модели, присваивая свойства одних героев к другим. Чтобы этого избежать, прибегают к лорбукам: к каждой записи присвается по триггеру (триггерам) и значению глубины: 0 — запись появляется в конце чата, 1-2-3-4 — запись появляется N-ной перед последними сообщениями.

# Формирование и отправка запроса:

В силу вступают технические ограничения. Gemini — пропиретарная модель, а Disco Elysium — очень противоречивая со стороны политических мировоззрений и поднимаемых тем игра, и её тематика подразумевает возможность восхваливать темы шовинизма, антифеминизма, и осуждать либерализм; для поддержания иммерсивных ответов приходится углубиться в принцип работы фильтра.

Фильтр Gemini срабатывает каждый раз, когда через него проходит блок текста, туда же входит стриминг токенов (который стал очень плавным на 2.x версиях, в качестве ответа может отсылаться до 2-3 токенов, что вызывает крайне много проверок классификатором). Сам классификатор при этом крайне неточный и очень быстрый, поэтому существует множество ложных срабатываний: Gemini вводил негативный bias во все политические варианты ответов и отказывался проводить игрока через темы убийства, которое расследуется. Его эффективность сводится к нулю на английском языке и на больших объёмах контекста, но засорение контекста или попытка его отравления негативно сказываются на эффективности модели в прозе.

# Выбор формата промпта.

С тех пор, как LLM стали хорошо работать со структурированным текстом, встал вопрос об оптимизации входных данных, так как естественный язык может быть слишком тяжёлым для восприятия модели. Используются разные запросы, со своими плюсами и минусами:
1. **Plain text**. Так как модель обучалась в первую очередь на восприятие естественного языка, *хорошо сформулированный* запрос будет выдавать более качественный результат. Сюда же могут входить и симбиозы, использующие вставки, сделанные в ``xml`` или ``markdown``. 
2. **"Интервью"**. Под эту категорию входит множество нечётких реализаций, которые способны поддерживать чётко прописанного персонажа лучшим образом. К сожалению, почти ни одна модель не способна воспринимать формат интервью в качестве направлений, а не прямых указаний к форме и виду текста. Возникают "эхо" и зацикливания форматирования, когда ответ модели начинает жёстко подстраиваться под формат интервью или переданный пример общения. Gemini по личному опыту очень легко поддаётся этой особенности, что делает применение подобной стратегии крайне ограниченным.
- ```Как зовут этого персонажа? \nАлекс. \n\nВ чём он хорош? В вышивании, вязании и «решении проблем». ```
3. **Markdown**. Один из наиболее популярных и легковоспринимаемых стилей текста для модели, так как используется в README файлах, и оттого легко воспринимается структурно моделью. Хорошо подходит для коротких и чётко структурированных описаний, по личному тестированию — разделение на главы плохо сказывается на запоминаемости принадлежности признаков.
- ```# Character \ - Name: Alex \ ## Traits \ - Knitting, sewing, problem-solving```
4. **XML**. Один из наиболее структурированных форматов файлов, но в этом и его проблема; модель очень часто воспринимает содержимое каждого xml-тега как самодостаточную единицу информации, и теряет из виду, к какому большему блоку оно принадлежит. Существует большая проблема с объёмностью формата. Применение xml-тегов способно "разъесть" информацию о персонаже на ~30-40%. 
- ```<character name="Alex><traits><trait>Knitting</trait><trait>Sewing</trait><trait>problem-solving</trait></traits></character>```
5. **W++**. Формат, ставший очень популярным на заре текстовых моделей GPT-3 и NovelAI. Помимо того, что сам его вид сильно ограничивает пространство в описании, до 60% всех токенов могут принадлежать самому форматированию вместо полезной информации.
- ```[{Character("Alex"){traits("knitting" + "sewing" + "problem-solving")}}]```
6. **JSON**. Субъективно, один из лучших форматов, которые можно использовать для записей в World Info. В condense формате запись на ~700 токенов «разрастается» всего лишь до ~750. Могу лишь предположить, что дело в уникальных знаках для целых блоков и для содержимого в них. 
- ```{"name": "Alex", "traits": "knitting", "sewing", "problem-solving"}```


In [25]:
# World_info

import json
import uuid

# Загрузка данных из файла
with open('results.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

converted = []

for name, details in data.items():
    # Формируем trigger
    trigger_values = []
    for key in ["Attribute", "Skill name"]:
        value = details.get(key)
        if isinstance(value, list):
            trigger_values.extend(value)
        elif value is not None:
            trigger_values.append(value)

    entry_dict = {
        k: v for k, v in details.items()
    }

    # Преобразуем entry в строку JSON
    entry_str = json.dumps(entry_dict, ensure_ascii=False)

    # Формируем запись
    converted.append({
        "uuid": str(uuid.uuid4()),
        "trigger": trigger_values,
        "entry": entry_str
    })

# Сохраняем результат
with open('world_info.json', 'w', encoding='utf-8') as f:
    json.dump(converted, f, ensure_ascii=False, indent=2)

In [ ]:
import json
import re

last_message = 'I wake up and don\'t know where I am.'

# Загрузка и сортировка prompt'ов
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

prompts = sorted(data["prompts"], key=lambda x: x["id"])
chat_history = []
log_entries = []

# Обработка prompt'ов с учётом injection_depth
for p in prompts:
    content_substituted = substitute_variables(p["content"])
    escaped_content = escape(content_substituted)
    entry = {"role": p["role"], "content": escaped_content}

    depth = p["injection_depth"]
    if depth == "pre-fill" and p["role"] == "model":
        chat_history.append(entry)
    elif isinstance(depth, int):
        index = max(0, len(chat_history) - depth)
        chat_history.insert(index, entry)

# Добавление последнего пользовательского сообщения
chat_history.append({"role": "user", "content": escape(last_message)})

# Формирование финального текста
final_prompt_lines = []
for msg in chat_history:
    lines = msg["content"].split("\\n")
    joined = ' + '.join(f'"{line}\\n"' if j < len(lines) -
                        1 else f'"{line}"' for j, line in enumerate(lines))
    final_prompt_lines.append(f'{msg["role"].capitalize()}) ' + joined)

final_prompt = "\n".join(final_prompt_lines)

# # Лог финального промпта
# print("Final prompt:\n", final_prompt)


In [ ]:
import random
from pathlib import Path

history_path     = Path("history.json")
world_info_path  = Path("world_info.json")
json_path        = Path("prompt.json")
hide_blocks      = ['cyoa', 'thought', 'think']

persona = "[{{user}} is a 25-yo graduatee of City-State Hong-Kong Militia School of Revachol. For his far more than average citizen of this Isola, he was usually called Big Mark. This name sticked with him with how many people unintentionally came up with the same name. Now he's on his duty on Precinct 41, covering Kowloon Walled City, being a sattelite-officer of Harrier De Bois.]" + "He has a plethora of useful skills:" + persona_skills + "]"
variables = {
    "user": "Marc",
    "persona": persona,
    "description": "Lieutenant double-yefreitor Harrier \"Harry\" Du Bois. Alcoholic and drug addict, but one very worthy detective. Lives in Hong Kong as a former descendant detective from Britain.",
    "scenario": "This is a fused world of Disco Elysium and out reality as of 1980's. Ocean is covered by Pale, and every continent is called an isola. A satteline-officer of Hong Kong, annexed by Taiwan and kept with a slippery peace deal, is out on his first day to solve the case of seemingly policially incline murder. He's assisting Harrier De Bois, a masterful detective with more than 200 solved cases."
}

events = [
    "Introduce a sudden change in scenery in the next reply.</guidelines>",
    "There must be a plot twist in the next reply.</guidelines>"
    "There's something behind the corner in the next reply.</guidelines>"
    "Suddenly, one of the characters feels like's going to throw up. Write it in the next reply.</guidelines>"
    "</guidelines>",
    "</guidelines>",
    "</guidelines>",
    "</guidelines>",
    "</guidelines>",
    "</guidelines>",
    "</guidelines>",
    "</guidelines>",
    "</guidelines>",
    "</guidelines>",
    "</guidelines>",
]



# Утилиты
def escape(text):
    return json.dumps(text)[1:-1]


def substitute_variables(text: str) -> str:
    """{{var}} и {{random:a,b,c}}"""
    text = re.sub(r"\{\{random:([^}]+)\}\}",
                  lambda m: random.choice([x.strip()
                                          for x in m.group(1).split(',')]),
                  text)
    return re.sub(r"\{\{(\w+)\}\}",
                  lambda m: variables.get(m.group(1), m.group(0)),
                  text)


def strip_xml_blocks(text: str, tags):
    for tag in tags:
        # удаляем многострочные блоки <tag> … </tag>
        patt = rf'<{tag}>(?:.|\n)*?</{tag}>'
        text = re.sub(patt, '', text, flags=re.DOTALL)
    return text


def load_world_info():
    return json.loads(Path(world_info_path).read_text('utf-8')) if world_info_path.exists() else []


def apply_world_info(messages, world_info, window=2):
    recent = " ".join(m["content"] for m in messages[-window:])
    used = {m["content"] for m in messages if m.get("wi_entry")}
    inserts = []
    for wi in world_info:
        triggers = [t for t in wi.get("trigger", []) if t.strip()]
        if triggers and any(t in recent for t in triggers):
            if wi["entry"] not in used:
                inserts.append(
                    {"role": "user", "content": wi["entry"], "wi_entry": True}
                )
                used.add(wi["entry"])
    if inserts:
        messages[-window:-window] = inserts
    return messages


def build_prompt(messages):
    parts = []
    for m in messages:
        safe = json.dumps(m["content"])[1:-1] # экранируем только для prompt
        parts.append(f'{m["role"].capitalize()}) "{safe}"')
    return "\n".join(parts)


if history_path.exists():
    messages = json.loads(history_path.read_text('utf-8'))
else:
    messages = []

messages = [m for m in messages if m.get("role") in ("user", "model")]

prompts = sorted(
    json.loads(Path(json_path).read_text('utf-8'))["prompts"],
    key=lambda x: x["id"]
)

print("Введите сообщение (или /exit для выхода):")

while True:
    user_input = input("User: ")
    if user_input.strip().lower() == "/exit":
        print("Диалог завершён.")
        break

    messages.append({"role": "user", "content": user_input})

    tmp_msgs = []

    for p in prompts:
        content = substitute_variables(p["content"])
        tmp_msgs.append({"role": p["role"], "content": content})

    tmp_msgs.extend(messages)

    for m in tmp_msgs[:-2]:
        m["content"] = strip_xml_blocks(m["content"], hide_blocks)

    world_info = load_world_info()
    tmp_msgs = apply_world_info(tmp_msgs, world_info)

    final_prompt = build_prompt(tmp_msgs)

    response = client.models.generate_content(
        model=GEMINI_MODEL, contents=final_prompt,
        config=types.GenerateContentConfig(
            max_output_tokens=4000, temperature=1.0, top_k=1, top_p=0.9)
    )
    ai_reply = response.text
    print(f"AI : {ai_reply}\n")
    messages.append({"role": "model", "content": ai_reply})
    history_path.write_text(json.dumps(messages, indent=2, ensure_ascii=False))

Введите сообщение (или /exit для выхода):
AI : Сознание возвращается медленно, неохотно, принося с собой раскалывающую головную боль и отвратительный вкус во рту, будто там переночевала стая дохлых крыс. Веки тяжелые, как свинцовые гири. Открыв глаза, Марк видит перед собой обшарпанный потолок с большим влажным пятном неопределенной формы. Воздух в комнате спертый, пахнет дешевым табаком, пролитым алкоголем и чем-то еще, неуловимо тошнотворным.

Марк лежит на неудобной, прогнувшейся кровати с грязным, скомканным постельным бельем. Рядом, на полу, валяется пустая бутылка из-под какого-то местного пойла. Одежда – вчерашняя, измятая – все еще на нем. Комната маленькая, заставленная разномастной рухлядью: шаткий стол у окна, заваленный окурками и пустыми пачками сигарет, стул с ободранной обивкой, старый, неработающий вентилятор на тумбочке. Грязно-желтые обои местами отошли от стен, обнажая бетон. За единственным окном, задернутым пыльной шторой, едва пробивается тусклый утренний свет и с